In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [101]:
spiro = pd.read_excel('spiro.xlsx')  #набор данных о анализах, имт и болезнях пациентов
spiro

,Организация/профгруппа,Пол,Возраст,"Рост, см","Вес, кг",ИМТ,"Обх.талии, см",Индекс курения,ЧСС (уд/мин.),АДС(мм рт. ст),...,"Гамма-глутамилтрансфераза, Ед./л",Индекс атерогенности,Норма,Рестрикция,Об и рес,Обструкция,Нарушение без уточнений,Заболевание,Наруш,Код_наруш
0,ЕПК,0.0,48.0,176.0,98.0,31.637397,110.0,30.0,80.0,130.0,...,32.0,5.5,1,0,0,0,0,NaN,норма,0
1,ЕПК,0.0,55.0,181.0,112.0,34.186991,113.0,0.0,60.0,150.0,...,26.8,3.7,1,0,0,0,0,NaN,норма,0
2,ЕПК,0.0,43.0,183.0,86.0,25.680074,97.0,0.0,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
3,ЕПК,0.0,55.0,174.0,70.0,23.120624,83.0,0.0,78.0,120.0,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
4,ЕПК,0.0,58.0,182.0,77.0,23.245985,96.0,0.0,72.0,110.0,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,ЕПК,0.0,72.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
1487,ЕПК,0.0,62.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,1,0,0,0,0,NaN,норма,0
1488,ЕПК,1.0,30.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0,1,0,0,1,NaN,рестрикция,1
1489,ЕПК,0.0,47.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0,0,0,1,1,NaN,обструкция,3


In [102]:
spiro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 42 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Организация/профгруппа                  1490 non-null   object 
 1   Пол                                     1490 non-null   float64
 2   Возраст                                 1490 non-null   float64
 3   Рост, см                                715 non-null    float64
 4   Вес, кг                                 715 non-null    float64
 5   ИМТ                                     715 non-null    float64
 6   Обх.талии, см                           715 non-null    float64
 7   Индекс курения                          1490 non-null   float64
 8   ЧСС (уд/мин.)                           622 non-null    float64
 9   АДС(мм рт. ст)                          622 non-null    float64
 10  АДД(мм рт. ст)                          622 non-null    floa

In [103]:
spiro

,Организация/профгруппа,Пол,Возраст,"Рост, см","Вес, кг",ИМТ,"Обх.талии, см",Индекс курения,ЧСС (уд/мин.),АДС(мм рт. ст),...,"Гамма-глутамилтрансфераза, Ед./л",Индекс атерогенности,Норма,Рестрикция,Об и рес,Обструкция,Нарушение без уточнений,Заболевание,Наруш,Код_наруш
0,ЕПК,0.0,48.0,176.0,98.0,31.637397,110.0,30.0,80.0,130.0,...,32.0,5.5,1,0,0,0,0,NaN,норма,0
1,ЕПК,0.0,55.0,181.0,112.0,34.186991,113.0,0.0,60.0,150.0,...,26.8,3.7,1,0,0,0,0,NaN,норма,0
2,ЕПК,0.0,43.0,183.0,86.0,25.680074,97.0,0.0,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
3,ЕПК,0.0,55.0,174.0,70.0,23.120624,83.0,0.0,78.0,120.0,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
4,ЕПК,0.0,58.0,182.0,77.0,23.245985,96.0,0.0,72.0,110.0,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1486,ЕПК,0.0,72.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,норма,0
1487,ЕПК,0.0,62.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,1,0,0,0,0,NaN,норма,0
1488,ЕПК,1.0,30.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0,1,0,0,1,NaN,рестрикция,1
1489,ЕПК,0.0,47.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0,0,0,1,1,NaN,обструкция,3


In [104]:
spiro_clear = spiro.drop(['Альбумин, г/л', 'ЛПВП, ммоль/л', 'Мочевина, ммоль/л', 'Белок общий, г/л', 'Норма', 'Рестрикция', 'Об и рес', 'Обструкция', 'Заболевание', 'Наруш', 'Код_наруш'], axis=1).dropna()

In [105]:
spiro_clear

,Организация/профгруппа,Пол,Возраст,"Рост, см","Вес, кг",ИМТ,"Обх.талии, см",Индекс курения,ЧСС (уд/мин.),АДС(мм рт. ст),...,"Глюкоза, ммоль/л","Креатинин, мкмоль/л","ЛПНП, ммоль/л","Мочевая кислота, мкмоль/л","Триглицериды, ммоль/л","Общий холестерин, ммоль/л","Щелочная фосфатаза, Ед/л","Гамма-глутамилтрансфераза, Ед./л",Индекс атерогенности,Нарушение без уточнений
0,ЕПК,0.0,48.0,176.0,98.0,31.637397,110.0,30.0,80.0,130.0,...,5.81,98.8,3.90,400.0,2.17,5.83,155.0,32.0,5.50,0
1,ЕПК,0.0,55.0,181.0,112.0,34.186991,113.0,0.0,60.0,150.0,...,5.28,113.6,3.46,335.0,1.30,5.32,187.0,26.8,3.70,0
8,ЕПК,0.0,27.0,176.0,143.0,46.164773,125.0,0.0,60.0,150.0,...,4.68,99.3,3.29,512.0,1.41,5.28,132.0,21.7,3.00,0
12,ЕПК,0.0,44.0,168.0,81.0,28.698980,104.0,0.0,78.0,110.0,...,5.73,103.5,4.31,333.0,2.00,6.44,146.0,23.0,4.40,0
16,ЕПК,0.0,57.0,171.0,80.0,27.358845,96.0,0.0,57.0,150.0,...,6.44,130.2,2.97,485.0,0.81,4.43,131.0,15.6,2.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,НПЗ,0.0,46.0,180.0,83.0,25.617284,98.0,30.0,62.0,160.0,...,4.70,109.0,3.55,376.0,2.23,5.15,167.0,38.8,3.02,0
705,НПЗ,0.0,46.0,167.0,92.0,32.987916,109.0,20.0,77.0,150.0,...,10.63,83.2,2.12,454.0,11.30,8.68,273.0,94.5,2.13,0
706,НПЗ,0.0,56.0,180.0,92.0,28.395062,108.0,0.0,91.0,130.0,...,5.08,115.0,3.98,197.0,1.65,5.88,154.0,30.8,3.11,0
708,НПЗ,0.0,47.0,185.0,92.0,26.880935,103.0,25.0,67.0,120.0,...,4.91,116.4,3.28,321.0,1.20,5.81,177.0,82.1,2.26,0


In [106]:
spiro_clear['Организация/профгруппа'].unique()

array(['ЕПК', 'НПЗ', 'Лысогорская птицефабрика',
       'Симоновская птицефабрика'], dtype=object)

In [107]:

org_dict = {
    'ЕПК': 1,
    'НПЗ': 2,
    'Лысогорская птицефабрика': 3,
    'Симоновская птицефабрика': 4
}

spiro_clear['Организация/профгруппа'] = spiro_clear['Организация/профгруппа'].replace(org_dict).astype(int)

<ipython-input-107-539b2e0cc3ad>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spiro_clear['Организация/профгруппа'] = spiro_clear['Организация/профгруппа'].replace(org_dict).astype(int)


In [108]:
spiro_clear['Организация/профгруппа'].unique()

array([1, 2, 3, 4])

In [110]:
X = spiro_clear.drop(['Нарушение без уточнений'], axis=1)
y = spiro_clear['Нарушение без уточнений']

In [111]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучение модели
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [116]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Оцениваем качество
train_predictions = rf_classifier.predict(X_train)
test_predictions = rf_classifier.predict(X_test)

print(f"Точность на обучающей выборке: {accuracy_score(y_train, train_predictions):.3f}")
print(f"Точность на тестовой выборке: {accuracy_score(y_test, test_predictions):.3f}")
print(f"AUC-ROC на тестовой выборке: {roc_auc_score(y_test, test_predictions):.3f}")

# Выводим важности признаков
feature_importances = rf_classifier.feature_importances_
print(f"\nВажности первых {len(feature_importances)} признаков:")
for i in range(len(feature_importances)):
    print(f"Признак {i}: {feature_importances[i]:.3f}")

Точность на обучающей выборке: 1.000
Точность на тестовой выборке: 0.723
AUC-ROC на тестовой выборке: 0.602

Важности первых 30 признаков:
Признак 0: 0.009
Признак 1: 0.022
Признак 2: 0.054
Признак 3: 0.067
Признак 4: 0.042
Признак 5: 0.045
Признак 6: 0.034
Признак 7: 0.021
Признак 8: 0.036
Признак 9: 0.020
Признак 10: 0.010
Признак 11: 0.003
Признак 12: 0.039
Признак 13: 0.054
Признак 14: 0.006
Признак 15: 0.000
Признак 16: 0.008
Признак 17: 0.051
Признак 18: 0.046
Признак 19: 0.032
Признак 20: 0.055
Признак 21: 0.035
Признак 22: 0.052
Признак 23: 0.037
Признак 24: 0.040
Признак 25: 0.038
Признак 26: 0.039
Признак 27: 0.038
Признак 28: 0.030
Признак 29: 0.038
